In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn import preprocessing

In [2]:
# Set a seed for reproducibility
seed_value = 42
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)

## Data Loading

In [3]:
df_bank_a = pd.read_csv('../data/BankA.csv')
df_bank_b = pd.read_csv('../data/BankB.csv')
df_bank_c = pd.read_csv('../data/BankC.csv')

In [4]:
df_all = pd.concat([df_bank_a, df_bank_b, df_bank_c])

## Data Preprocessing

In [5]:
# Strip all string values from the dataset
df = df_all.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

### Feature Columns

In [6]:
# Combine Never-worked and Without-pay into one category
df['workclass'] = df['workclass'].replace(['Never-worked', 'Without-pay'], 'Not-working')
df['workclass'] = df['workclass'].replace(['?', '*'], 'unknown')
df['workclass'].value_counts()

workclass
Private             546342
Self-emp-not-inc     66145
Local-gov            51137
unknown              47431
State-gov            34717
Self-emp-inc         27715
Federal-gov          25879
Not-working            633
Name: count, dtype: int64

In [7]:
# Combine Married-civ-spouse and Married-AF-spouse into one category
df['marital-status'] = df['marital-status'].replace(['Married-civ-spouse', 'Married-AF-spouse'], 'Married')
df['marital-status'].value_counts()

marital-status
Married                  368820
Never-married            250510
Divorced                 110459
Widowed                   34203
Separated                 25566
Married-spouse-absent     10441
Name: count, dtype: int64

In [8]:
# Replace occupation by 4 categories (low, medium, high, unknown)
df['occupation'] = df['occupation'].replace(['Exec-managerial', 'Prof-specialty'], 'high')
df['occupation'] = df['occupation'].replace(['Armed-Forces', 'Protective-serv', 'Tech-support', 'Sales', 'Craft-repair', 'Transport-moving'], 'medium')
df['occupation'] = df['occupation'].replace(['Adm-clerical', 'Machine-op-inspct', 'Farming-fishing', 'Handlers-cleaners', 'Other-service', 'Priv-house-serv'], 'low')
df['occupation'] = df['occupation'].replace(['?', '*'], 'unknown')
df['occupation'].value_counts()

occupation
low        281223
medium     259651
high       211330
unknown     47795
Name: count, dtype: int64

In [9]:
# Combine Husband and Wife into one category
df['relationship'] = df['relationship'].replace(['Husband', 'Wife'], 'Parent')
df['relationship'].value_counts()

relationship
Parent            362767
Not-in-family     212898
Own-child         119123
Unmarried          80532
Other-relative     24679
Name: count, dtype: int64

In [10]:
# Map native-country to continents
df['native-country'] = df['native-country'].str.strip()
df['native-country'] = df['native-country'].replace(['United-States', 'Puerto-Rico', 'Canada', 'Outlying-US(Guam-USVI-etc)', 'Cuba', 'Jamaica', 'Mexico', 'Dominican-Republic', 'El-Salvador', 'Guatemala', 'Haiti', 'Honduras', 'Nicaragua', 'Trinadad&Tobago', 'Peru', 'Ecuador', 'Columbia', 'Honduras', 'Haiti', 'Guatemala', 'El-Salvador', 'Dominican-Republic', 'Columbia', 'Ecuador', 'Peru', 'Jamaica', 'Mexico', 'Puerto-Rico', 'Cuba', 'Outlying-US(Guam-USVI-etc)', 'Canada', 'United-States'], 'North-America')
df['native-country'] = df['native-country'].replace(['Germany', 'England', 'Italy', 'Poland', 'Portugal', 'Ireland', 'France', 'Yugoslavia', 'Scotland', 'Greece', 'Hungary', 'Holand-Netherlands'], 'Europe')
df['native-country'] = df['native-country'].replace(['Philippines', 'India', 'China', 'Japan', 'Vietnam', 'Taiwan', 'Iran', 'Thailand', 'Hong', 'Cambodia', 'Laos'], 'Asia')
df['native-country'] = df['native-country'].replace(['South', 'Columbia', 'Ecuador', 'Peru'], 'South-America')
df['native-country'] = df['native-country'].replace(['Trinadad&Tobago', 'Honduras', 'Haiti', 'Guatemala', 'El-Salvador', 'Dominican-Republic', 'Columbia', 'Ecuador', 'Peru'], 'Central-America')
df['native-country'] = df['native-country'].replace(['?', '*'], 'Unknown')
df['native-country'].value_counts()

native-country
North-America    753935
Asia              14694
Unknown           14390
Europe            14273
South-America      2707
Name: count, dtype: int64

In [11]:
df['education'] = df['education'].replace(['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th'], 'school')
df['education'] = df['education'].replace(['Assoc-voc', 'Assoc-acdm', 'Prof-school', 'Some-college'], 'higher')
df['education'].value_counts()

education
HS-grad      258661
higher       241834
Bachelors    133796
school       110209
Masters       45697
Doctorate      9802
Name: count, dtype: int64

In [12]:
df['race'] = df['race'].replace(['Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], 'Other')
df['race'].value_counts()

race
White    686196
Other    113803
Name: count, dtype: int64

In [13]:
# Convert age in bins of 10 years
df['age'] = pd.cut(df['age'], bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '>90'])
df['age'] = df['age'].astype('object')
df['age'].value_counts()

age
20-30    199748
30-40    187599
40-50    168168
50-60    107704
0-20      64609
60-70     51995
70-80     15668
80-90      4508
Name: count, dtype: int64

In [14]:
# Drop the fnlwgt column
df.drop(['fnlwgt'], axis=1, inplace=True)

# Drop the education column
# df.drop(['education'], axis=1, inplace=True)

# Drop the capital-gain column
df.drop(['capital-gain'], axis=1, inplace=True)

# Drop the capital-loss column
df.drop(['capital-loss'], axis=1, inplace=True)

# Drop the gender column
df.drop(['gender'], axis=1, inplace=True)

# Drop the age column
# df.drop(['age'], axis=1, inplace=True)

# Drop the race column
df.drop('race', axis=1, inplace=True)

### Target Column

In [15]:
# replace income by 0 and 1
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})
df['income'].value_counts()

income
0    601449
1    198550
Name: count, dtype: int64

In [16]:
categorical_columns = df.select_dtypes(include=['object']).columns
 
for column in df[categorical_columns].columns:
    print(f"Column: {column}")
    print(df[column].value_counts())
    print("\n" + "="*30 + "\n")

# Print also the numberical columns, categorize them into bins of 6
numerical_columns = df.select_dtypes(include=['int64']).columns

for column in df[numerical_columns].columns:
    print(f"Column: {column}")
    print(df[column].value_counts(bins=6))
    print("\n" + "="*30 + "\n")

Column: institute
institute
Bank B    403240
Bank A    226164
Bank C    170595
Name: count, dtype: int64


Column: age
age
20-30    199748
30-40    187599
40-50    168168
50-60    107704
0-20      64609
60-70     51995
70-80     15668
80-90      4508
Name: count, dtype: int64


Column: workclass
workclass
Private             546342
Self-emp-not-inc     66145
Local-gov            51137
unknown              47431
State-gov            34717
Self-emp-inc         27715
Federal-gov          25879
Not-working            633
Name: count, dtype: int64


Column: education
education
HS-grad      258661
higher       241834
Bachelors    133796
school       110209
Masters       45697
Doctorate      9802
Name: count, dtype: int64


Column: marital-status
marital-status
Married                  368820
Never-married            250510
Divorced                 110459
Widowed                   34203
Separated                 25566
Married-spouse-absent     10441
Name: count, dtype: int64


Column: occupat

In [17]:
# One-hot encode the categorical columns
df = pd.get_dummies(df, columns=categorical_columns)

## Model Training

### Train-Test Split

In [18]:
# Drop institutes columns
df_all = df.drop(['institute_Bank A', 'institute_Bank B', 'institute_Bank C'], axis=1)
df_bank_a = df[df['institute_Bank A'] == 1].drop(['institute_Bank A', 'institute_Bank B', 'institute_Bank C'], axis=1)
df_bank_b = df[df['institute_Bank B'] == 1].drop(['institute_Bank A', 'institute_Bank B', 'institute_Bank C'], axis=1)
df_bank_c = df[df['institute_Bank C'] == 1].drop(['institute_Bank A', 'institute_Bank B', 'institute_Bank C'], axis=1)

In [19]:
# number of rows in each dataset
print(f"Number of rows in df_all:    {len(df_all)}")
print(f"Number of rows in df_bank_a: {len(df_bank_a)}")
print(f"Number of rows in df_bank_b: {len(df_bank_b)}")
print(f"Number of rows in df_bank_c: {len(df_bank_c)}")

Number of rows in df_all:    799999
Number of rows in df_bank_a: 226164
Number of rows in df_bank_b: 403240
Number of rows in df_bank_c: 170595


In [20]:
# Create a test set that contains 20% of the data from each bank
df_bank_a_test = df_bank_a.sample(frac=0.2, random_state=seed_value)
df_bank_b_test = df_bank_b.sample(frac=0.2, random_state=seed_value)
df_bank_c_test = df_bank_c.sample(frac=0.2, random_state=seed_value)

# Create a training set that contains the remaining 80% of the data from each bank
df_bank_a_train = df_bank_a.drop(df_bank_a_test.index)
df_bank_b_train = df_bank_b.drop(df_bank_b_test.index)
df_bank_c_train = df_bank_c.drop(df_bank_c_test.index)

# Combine the training sets into one training set
df_train = pd.concat([df_bank_a_train, df_bank_b_train, df_bank_c_train])

# Combine the test sets into one test set
df_test = pd.concat([df_bank_a_test, df_bank_b_test, df_bank_c_test])

### Model Training

In [21]:
# Train a decision tree classifier with grid search to find the best hyperparameters
X_train = df_train.drop('income', axis=1)
y_train = df_train['income']

X_test = df_test.drop('income', axis=1)
y_test = df_test['income']

In [22]:
# Find the best hyperparameters for a decision tree classifier

# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=seed_value)

# Define the grid search parameters
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10, 12, 14, 16],
    'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16],
    'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16],
    'max_features': [2, 4, 6, 8, 10, 12, 14, 16]
}

# Create a grid search object
from sklearn.model_selection import RandomizedSearchCV
grid_search = RandomizedSearchCV(estimator=clf, n_iter=30, param_distributions=param_grid, cv=5, n_jobs=-1, verbose=0)

# Train the grid search object to find the best model and the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

{'min_samples_split': 16, 'min_samples_leaf': 6, 'max_features': 6, 'max_depth': 16, 'criterion': 'entropy'}
0.8150747281717046
DecisionTreeClassifier(criterion='entropy', max_depth=16, max_features=6,
                       min_samples_leaf=6, min_samples_split=16,
                       random_state=42)


#### Train a model with all data from all banks

In [23]:
# Train a decision tree classifier with the best hyperparameters
model = grid_search.best_estimator_
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=16, max_features=6,
                       min_samples_leaf=6, min_samples_split=16,
                       random_state=42)

In [24]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
accuracy_global_model = accuracy_score(y_test, y_pred)
f1_global_model = f1_score(y_test, y_pred)
kappa_global_model = cohen_kappa_score(y_test, y_pred)

In [25]:
# Evaluate the model on each banks test set
y_pred_bank_a = model.predict(df_bank_a_test.drop('income', axis=1))
accuracy_bank_a_global_model = accuracy_score(df_bank_a_test['income'], y_pred_bank_a)
f1_bank_a_global_model = f1_score(df_bank_a_test['income'], y_pred_bank_a)
kappa_bank_a_global_model = cohen_kappa_score(df_bank_a_test['income'], y_pred_bank_a)

y_pred_bank_b = model.predict(df_bank_b_test.drop('income', axis=1))
accuracy_bank_b_global_model = accuracy_score(df_bank_b_test['income'], y_pred_bank_b)
f1_bank_b_global_model = f1_score(df_bank_b_test['income'], y_pred_bank_b)
kappa_bank_b_global_model = cohen_kappa_score(df_bank_b_test['income'], y_pred_bank_b)

y_pred_bank_c = model.predict(df_bank_c_test.drop('income', axis=1))
accuracy_bank_c_global_model = accuracy_score(df_bank_c_test['income'], y_pred_bank_c)
f1_bank_c_global_model = f1_score(df_bank_c_test['income'], y_pred_bank_c)
kappa_bank_c_global_model = cohen_kappa_score(df_bank_c_test['income'], y_pred_bank_c)

#### Train three models, one for each bank

In [26]:
# Train model on Bank A data
X_train_bank_a = df_bank_a_train.drop('income', axis=1)
y_train_bank_a = df_bank_a_train['income']

X_test_bank_a = df_bank_a_test.drop('income', axis=1)
y_test_bank_a = df_bank_a_test['income']

model_bank_a = grid_search.best_estimator_
model_bank_a.fit(X_train_bank_a, y_train_bank_a)

# Train model on Bank B data
X_train_bank_b = df_bank_b_train.drop('income', axis=1)
y_train_bank_b = df_bank_b_train['income']

X_test_bank_b = df_bank_b_test.drop('income', axis=1)
y_test_bank_b = df_bank_b_test['income']

model_bank_b = grid_search.best_estimator_
model_bank_b.fit(X_train_bank_b, y_train_bank_b)

# Train model on Bank C data
X_train_bank_c = df_bank_c_train.drop('income', axis=1)
y_train_bank_c = df_bank_c_train['income']

X_test_bank_c = df_bank_c_test.drop('income', axis=1)
y_test_bank_c = df_bank_c_test['income']

model_bank_c = grid_search.best_estimator_
model_bank_c.fit(X_train_bank_c, y_train_bank_c)

DecisionTreeClassifier(criterion='entropy', max_depth=16, max_features=6,
                       min_samples_leaf=6, min_samples_split=16,
                       random_state=42)

In [27]:
# Evaluate each model on their own test set
y_pred_bank_a = model_bank_a.predict(X_test_bank_a)
accuracy_bank_a_local_model = accuracy_score(y_test_bank_a, y_pred_bank_a)
f1_bank_a_local_model = f1_score(y_test_bank_a, y_pred_bank_a)
kappa_bank_a_local_model = cohen_kappa_score(y_test_bank_a, y_pred_bank_a)

y_pred_bank_b = model_bank_b.predict(X_test_bank_b)
accuracy_bank_b_local_model = accuracy_score(y_test_bank_b, y_pred_bank_b)
f1_bank_b_local_model = f1_score(y_test_bank_b, y_pred_bank_b)
kappa_bank_b_local_model = cohen_kappa_score(y_test_bank_b, y_pred_bank_b)

y_pred_bank_c = model_bank_c.predict(X_test_bank_c)
accuracy_bank_c_local_model = accuracy_score(y_test_bank_c, y_pred_bank_c)
f1_bank_c_local_model = f1_score(y_test_bank_c, y_pred_bank_c)
kappa_bank_c_local_model = cohen_kappa_score(y_test_bank_c, y_pred_bank_c)

In [28]:
# Compare the accuracies of the models

print("Accuracies:")
print()
print("global model, global testset: ", round(accuracy_global_model, 4))
print()
print("global model, bank A testset: ", round(accuracy_bank_a_global_model, 4))
print("global model, bank B testset: ", round(accuracy_bank_b_global_model, 4))
print("global model, bank C testset: ", round(accuracy_bank_c_global_model, 4))
print()
print("bank A model, bank A testset: ", round(accuracy_bank_a_local_model, 4))
print("bank B model, bank B testset: ", round(accuracy_bank_b_local_model, 4))
print("bank C model, bank C testset: ", round(accuracy_bank_c_local_model, 4))

Accuracies:

global model, global testset:  0.8252

global model, bank A testset:  0.7978
global model, bank B testset:  0.8243
global model, bank C testset:  0.8639

bank A model, bank A testset:  0.7908
bank B model, bank B testset:  0.823
bank C model, bank C testset:  0.8628


In [29]:
# Create a dataframe with the accuracies
df_metrics = pd.DataFrame({
    'model': ['global model', 'global model', 'global model', 'bank A model', 'bank B model', 'bank C model'],
    'testset': ['global testset', 'bank A testset', 'bank B testset', 'bank A testset', 'bank B testset', 'bank C testset'],
    'accuracy': [accuracy_global_model, accuracy_bank_a_global_model, accuracy_bank_b_global_model, accuracy_bank_a_local_model, accuracy_bank_b_local_model, accuracy_bank_c_local_model],
    'f1 score': [f1_global_model, f1_bank_a_global_model, f1_bank_b_global_model, f1_bank_a_local_model, f1_bank_b_local_model, f1_bank_c_local_model],
    'kappa': [kappa_global_model, kappa_bank_a_global_model, kappa_bank_b_global_model, kappa_bank_a_local_model, kappa_bank_b_local_model, kappa_bank_c_local_model]
})
df_metrics

,model,testset,accuracy,f1 score,kappa
0,global model,global testset,0.825231,0.591537,0.483677
1,global model,bank A testset,0.797758,0.718176,0.560604
2,global model,bank B testset,0.824298,0.473430,0.377096
3,bank A model,bank A testset,0.790772,0.713160,0.548501
4,bank B model,bank B testset,0.822971,0.494637,0.393009
5,bank C model,bank C testset,0.862833,0.502022,0.427316


Erkenntnisse:  
1) "sensible" Daten 'gender', 'race' kann man weglassen, und trotzdem gute Modelle (> 80% Accuracy) trainieren
  
2) Der Accuracy-Vergleich zeigt, dass die Modelle für die einzelnen Banken minimal besser sind als das Modell für alle Banken zusammen. Da sie aber nur MINIMAL besser sind, deutet das darauf hin, dass der Aufwand von Federated Learning sich nicht lohnt. Jede Bank sollte ihr eigenes Modell trainieren, denn die Modellaggregation mit Modelle anderer Banken würde wahrscheinlich die Performance nur minimal verbessern.